Identify popular feature layers (based on item count), recently updated layers, and feature layers that use a lot of database storage

In [1]:
import datetime as dt
from pytz import timezone
from dateutil.relativedelta import relativedelta

from arcgis.gis import GIS

In [2]:
gis = GIS(profile="your_admin_profile", verify_cert=False)

## Popular feature layers based on number of views

### Create datetime objects to represent time ranges

In [3]:
# represent local timezone information
pst_tz = timezone("America/Los_Angeles")

In [4]:
# construct datetime objects for time range queries
jan27 = dt.datetime.now()
six_mos = relativedelta(months=6)
jul27 = jan27 - six_mos

# construct millisecond timestamps for searching
jan27_qy = int(jan27.timestamp()*1000)
jul27_qy = int(jul27.timestamp()*1000)

In [5]:
fl_6mos = gis.content.search(query=f"created: [{jul27_qy} TO {jan27_qy}]",
                             item_type="Feature Layer",
                             sort_field="numViews",
                             sort_order="desc",
                             max_items=500)
len(fl_6mos)

39

In [8]:
for flyr in fl_6mos:
    print(f"{flyr.title[:56]:58}{flyr.numViews}")

Community Analyst Test MLF Layers                         443
GeoPlanner MLF Test - Project Feature Service             421
PointLinePoly                                             340
Boston Public View                                        264
Hospitals                                                 54
Lines for Joining                                         25
Hospitals_CarstenView                                     22
Redlands Points with Many Characters                      17
LineDomain                                                16
Hospitals_MarianneView                                    13
Polygons                                                  8
crash2019                                                 7
City of Glasgow Recycling                                 6
Polygon Duplicate from fgdb                               6
crash_fgdb                                                6
Tussock_Within_Buffersbp                                  6
Campgrounds_Within_Rangebp

## Recently updated feature layers

In [9]:
# relative date objects
twenty_four_hrs = relativedelta(hours=24)
two_wks = relativedelta(weeks=2)
one_mo = relativedelta(months=1)
two_mo = relativedelta(months=2)

deltas = [twenty_four_hrs, two_wks, one_mo, two_mo]

# datetime objects 
hrs6_prev = jan27 - twenty_four_hrs
wks2_prev = jan27 - two_wks
mos1_prev = jan27 - one_mo
mos2_prev = jan27 - two_mo

time_pd_objs = [jan27 - rd for rd in deltas]

# corresponding millisecond values for platform queries
hrs6_prev_qy = int(hrs6_prev.timestamp()*1000)
wks2_prev_qy = int(wks2_prev.timestamp()*1000)
mos1_prev_qy = int(mos1_prev.timestamp()*1000)
mos2_prev_qy = int(mos2_prev.timestamp()*1000)

query_objs = [int(time_pd.timestamp()*1000) for time_pd in time_pd_objs]

In [11]:
time_ranges = ["day ago", "two weeks ago", "one month ago", "two_months_ago"]

In [20]:
for readable, date_ms in list(zip(time_ranges, query_objs)):
    print(f"{readable}\n{'-'*50}")
    flyr_list = gis.content.search(query=f"modified: [{date_ms} TO {jan27_qy}]",
                                   max_items=100,
                                   sort_field="type",
                                   sort_order="asc")
    print(f"{' ':3}{len(flyr_list)} items(s) modified\n{' '*3}{'='*20}")
    for flyr in flyr_list:
        print(f"{' ':5}{flyr.title[:38]:40}{flyr.type:30}{flyr.size}")
    print("\n")

day ago
--------------------------------------------------
   3 items(s) modified
     test                                    Dashboard                     0
     Elevation                               Web Mapping Application       1156
     Story Map Series                        Web Mapping Application       516


two weeks ago
--------------------------------------------------
   27 items(s) modified
     test                                    Dashboard                     0
     City of Glasgow Recycling_fieldworker   Feature Service               0
     City of Glasgow Recycling               Feature Service               12858
     CAO Project - patch testing Layers      Feature Service               0
     City of Glasgow Recycling_stakeholder   Feature Service               0
     City of Glasgow Recycling               Form                          65289
     CA Web App Suitability Analysis Data    Web Mapping Application       11
     CAO Project - patch testing Layers Fil

     CA Web App Comparison Reports Variable  Web Mapping Application       11
     CA Web App Custom Color Ramps           Web Mapping Application       11
     CA Web App Custom Color Picker Colors   Web Mapping Application       11
     Story Map Series                        Web Mapping Application       516
     f                                       Web Mapping Application       562
     CAO Project - patch testing Preference  Web Mapping Application       2
     CAO Project - patch testing Map Data    Web Mapping Application       76
     CA Web App Custom Symbols               Web Mapping Application       11
     CA Web App Void Analysis Data           Web Mapping Application       11




## Feature layers using above 1MB of database storage

In [21]:
mod_fl_mos2 = [fl 
               for fl in gis.content.search(query=f"modified: [{mos2_prev_qy} TO {jan27_qy}]",
                                            max_items=100)
               if fl.size/(1024*1024) > 1]

In [22]:
def get_mb(item):
    return item.size/(1024*1024)

In [23]:
mod_fl_mos2.sort(key=get_mb, reverse=True)

In [24]:
for item in mod_fl_mos2:
    print(f"{item.title[:35]:40}{item.type:20}{item.size/(1024 * 1024):.3} MB")

## Credit Usage

In [15]:
week_prev = mar19 - relativedelta(weeks=1)

In [17]:
credit_mgr = gis.admin.credits
credit_usg_week = credit_mgr.credit_usage(mar19, week_prev)

In [18]:
credit_usg_week

{'features': 482.77126599999997,
 'portal': 4.4237087100000005,
 'scene': 0.0217645416,
 'tiles': 0.388621652,
 'vectortiles': 0.77728984,
 'applogin': 0.0,
 'apploginprovider': 0.0}

In [19]:
total = sum(credit_usg_week.values())
total

488.3826507436

In [20]:
credit_pct = {k:(v/total*100) for k, v in credit_usg_week.items()}
credit_pct

{'features': 98.85102701026413,
 'portal': 0.9057874400871049,
 'scene': 0.004456452653848743,
 'tiles': 0.07957318946696688,
 'vectortiles': 0.15915590752794284,
 'applogin': 0.0,
 'apploginprovider': 0.0}

### User Usage Reports

In [21]:
usage_reports = gis.admin.usage_reports

In [22]:
user_usage = usage_reports.users(start_time=mar19, time_frame="60days")

In [23]:
def get_users_consuming_credits():
    usage_totals = []
    for usg_dict in user_usage['data']:
        if 'username' in usg_dict:
            total_usg = sum([float(ud[1]) for ud in usg_dict['credits']])
            if total_usg > 0.0:
                #print(f"{usg_dict['username']}\n{'-'*50}")
                #print(f"\t...used {total_usg} credits during this time")
                usage_totals.append({usg_dict['username']:total_usg})
    return usage_totals

In [24]:
user_credit_use = get_users_consuming_credits()
for usage_stats in user_credit_use:
    for user, credits in usage_stats.items():
        print(f"{user:40}{credits:>5.4}")

bstearman_uc                            13.37
deel4986@esri.com_onlineshowcase        0.008
dhrumil_demo                             0.92
edwi7454@esri.com_onlineshowcase         1.22
ianw_uc                                  0.01
jlias_DevSummit                          1.44
